# Required Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cma
from scipy.optimize import differential_evolution
from deap import base, creator, tools, algorithms
import random
from scipy.optimize import dual_annealing
import pyswarms as ps
from openpyxl import load_workbook


# Required Functions 

In [6]:
%run Optimization_RTK_Functions.ipynb

# Input

In [9]:
file_path = './CCW_event_9.xlsx'
data=pd.read_excel(file_path, skiprows=0)
rainfall= data.iloc[:,2].dropna().tolist() 
obs_rdii = data.iloc[:,1].tolist() 
delta_t = 600 #in sec ( for 10 min time step)
area_acres= 491.153 #23533.04 
total_rdii_period= (len(obs_rdii)-len(rainfall)) * delta_t  #seconds

### T and K Range 

In [13]:
# Allowed values for T (in seconds, 10-minute steps)
allowed_T1_values = [x * 60 for x in range(10, 121, 10)]  # 10 min to 2 hrs
allowed_T2_values = [x * 60 for x in range(20, 241, 10)]  # 20 min to 4 hrs
allowed_T3_values = [x * 60 for x in range(30, 421, 10)]  # 30 min to 7 hrs

# Allowed values for K (0.001 steps)
allowed_K1_values = [round(x, 3) for x in np.arange(1, 2.01, 0.001)]  # 1to 2
allowed_K2_values = [round(x, 3) for x in np.arange(2, 3.01, 0.001)]  # 2 to 3
allowed_K3_values = [round(x, 3) for x in np.arange(3.0, 7.01, 0.001)]  # 3 to 7


# CMA-ES Optimization 

### Objective Function defination 

In [16]:
def map_index_to_value(index, allowed_values):
    """Map a continuous index to the closest value in the allowed set."""
    index = int(round(index))  # Ensure it's an integer
    index = max(0, min(index, len(allowed_values) - 1))  # Clip to valid range
    return allowed_values[index]

def objective_function(params_flat, obs_rdii):
    """
    Objective function for optimization, calculates fitness and applies penalties.

    Args:
        params_flat (list): Flattened list of parameters (R, T, K indices).
        obs_rdii (array-like): Observed RDII values.

    Returns:
        float: Objective function value (fitness + penalties).
    """
    try:
        # Check for NaN or Inf in input
        if np.any(np.isnan(params_flat)) or np.any(np.isinf(params_flat)):
            print("Invalid CMA sample:", params_flat)
            return float('inf')

        # Unpack parameters
        R1, T1_idx, K1_idx, R2, T2_idx, K2_idx, R3, T3_idx, K3_idx = params_flat

        if R1 < 0 or R2 < 0 or R3 < 0:
            return float('inf')

        # Map discrete index to values
        T1 = map_index_to_value(T1_idx, allowed_T1_values)
        T2 = map_index_to_value(T2_idx, allowed_T2_values)
        T3 = map_index_to_value(T3_idx, allowed_T3_values)

        K1 = map_index_to_value(K1_idx, allowed_K1_values)
        K2 = map_index_to_value(K2_idx, allowed_K2_values)
        K3 = map_index_to_value(K3_idx, allowed_K3_values)

        # Check for hydrograph duration overflow (memory protection)
        max_duration = max(T1 + T1 * K1, T2 + T2 * K2, T3 + T3 * K3)
        if max_duration > total_rdii_period * 2:
            print("Hydrograph duration too large, skipping...")
            return float('inf')

        # Prepare parameter sets
        params = [(R1, T1, K1), (R2, T2, K2), (R3, T3, K3)]

        # Safe RDII calculation
        try:
            sim_rdii = RDII_calculation(params, delta_t, rainfall, area_acres)
            if sim_rdii is None or len(sim_rdii) == 0:
                print("Empty sim_rdii.")
                return float('inf')
            sim_rdii = np.array(sim_rdii)
            if np.any(np.isnan(sim_rdii)) or np.any(np.isinf(sim_rdii)):
                print("Invalid sim_rdii values.")
                return float('inf')
        except Exception as e:
            print("RDII calculation failed:", e)
            return float('inf')

        # Pad time series for equal length
        max_length = max(len(obs_rdii), len(sim_rdii))
        obs_rdii_padded = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), mode='constant')
        sim_rdii_padded = np.pad(sim_rdii, (0, max_length - len(sim_rdii)), mode='constant')

        # Compute fitness
        fitness_value = fitness(
            obs_rdii_padded,
            sim_rdii_padded,
            delta_t,
            weight_rmse=0.25,
            weight_r2=0.25,
            weight_pbias=0.25,
            weight_nse=0.25
        )

        # Penalty calculations
        Ro = R_calc(rainfall, obs_rdii, delta_t, area_acres)
        r_sum = R1 + R2 + R3

        if np.isnan(Ro) or np.isinf(Ro) or np.isnan(r_sum) or np.isinf(r_sum):
            print("Invalid Ro or R_sum.")
            return float('inf')

        penalty = 0

        # Equality constraint penalty (soft)
        penalty += 1e6 * (r_sum - Ro) ** 2

        # T1 < T2 < T3
        if not (T1 < T2 < T3):
            penalty += 1000

        # Ensure rising/receding limbs obey ordering
        if not (T1 + T1 * K1 < T2 + T2 * K2 < T3 + T3 * K3 <= total_rdii_period):
            penalty += 1000

        # Uncomment to debug:
        # print(f"R_sum={r_sum:.6f}, Ro={Ro:.6f}, Penalty={penalty:.1f}, Fitness={fitness_value:.4f}")

        return fitness_value + penalty

    except Exception as e:
        print("Exception in objective_function:", e)
        return float('inf')


In [18]:
# Initial guess for [R1, T1_idx, K1_idx, R2, T2_idx, K2_idx, R3, T3_idx, K3_idx]
initial_guess = [
    0.06, 2.0, 0.0,  # R1, T1_idx, K1_idx
    0.06, 4.0, 0.0,  # R2, T2_idx, K2_idx
    0.06, 3.0, 0.0   # R3, T3_idx, K3_idx
]

# Define standard deviations for each parameter
# R: Small values, T_idx: Larger range, K_idx: Medium range
stds = [0.001, 0.25, 5.0,  # For R1, T1_idx, K1_idx
        0.001, 0.25, 4.0,  # For R2, T2_idx, K2_idx
        0.001, 0.25, 5.0]  # For R3, T3_idx, K3_idx

# Sigma for parameters (adjusted for scaled indices)
sigma = 0.1

# Define bounds for R (continuous), T_idx, and K_idx
bounds_lower = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
bounds_upper = [1.0, len(allowed_T1_values) - 1, len(allowed_K1_values) - 1,
                1.0, len(allowed_T2_values) - 1, len(allowed_K2_values) - 1,
                1.0, len(allowed_T3_values) - 1, len(allowed_K3_values) - 1]

### Multilpe Run CMA-ES

In [ ]:
# Store results
all_results = []
all_fitness_scores = []

n_runs = 20  # Number of runs

for i in range(n_runs):
    print(f"Running CMA-ES iteration {i+1}/{n_runs}...")

    try:
        # Initialize CMA-ES
        es = cma.CMAEvolutionStrategy(
            initial_guess,
            sigma,
            {
                'CMA_stds': stds,
                'bounds': [bounds_lower, bounds_upper],
                'popsize': 10,
                'maxiter': 300,
                'CMA_diagonal': True,
                'seed': i
            }
        )

        # Optimize
        es.optimize(objective_function, args=(obs_rdii,))

        # Extract best result
        best_parameters = es.result.xbest
        best_fitness = es.result.fbest

        # Unpack and map
        R1, T1_idx, K1_idx, R2, T2_idx, K2_idx, R3, T3_idx, K3_idx = best_parameters
        T1 = map_index_to_value(T1_idx, allowed_T1_values)
        T2 = map_index_to_value(T2_idx, allowed_T2_values)
        T3 = map_index_to_value(T3_idx, allowed_T3_values)
        K1 = map_index_to_value(K1_idx, allowed_K1_values)
        K2 = map_index_to_value(K2_idx, allowed_K2_values)
        K3 = map_index_to_value(K3_idx, allowed_K3_values)

        best_params_actual = [(R1, T1, K1), (R2, T2, K2), (R3, T3, K3)]

        # Store results
        all_results.append(best_params_actual)
        all_fitness_scores.append(best_fitness)

    except Exception as e:
        print(f"Run {i+1} failed: {e}")
        continue

print("✅ Completed all CMA-ES runs.")


In [ ]:
all_params = all_results 

# Combine results into a DataFrame
columns = ['Run', 'R1', 'T1', 'K1', 'R2', 'T2', 'K2', 'R3', 'T3', 'K3', 'Fitness']
results_data = []

for run, (params, fitness) in enumerate(zip(all_results, all_fitness_scores), start=1):
    results_data.append([
        run,
        params[0][0], params[0][1], params[0][2],  # R1, T1, K1
        params[1][0], params[1][1], params[1][2],  # R2, T2, K2
        params[2][0], params[2][1], params[2][2],  # R3, T3, K3
        fitness  # Fitness score for the run
    ])

results_df = pd.DataFrame(results_data, columns=columns)

# # Save results to an Excel file
# filename = "cma_es_results_event5.xlsx"
# results_df.to_excel(filename, index=False)
# print(f"\nResults saved to {filename}")

# Display mean and standard deviation of parameters and fitness scores
mean_values = results_df.mean()
std_values = results_df.std()

print("\nMean values of parameters and fitness across runs:")
print(mean_values)

print("\nStandard deviation of parameters and fitness across runs:")
print(std_values)

### Saving RTK parameters

In [ ]:
# Define the Excel file name
excel_filename = "RTK_Parameters_all_algorithms_Ro_constraint_E9_CCW.xlsx"
sheet_name = "CMA_ES"

# Check if "Run" column exists, and add it only if not present
if "Run" not in results_df.columns:
    results_df.insert(0, "Run", range(1, len(results_df) + 1))

# Select the desired columns
export_columns = ["Run", "R1", "T1", "K1", "R2", "T2", "K2", "R3", "T3", "K3"]

# Check if the Excel file exists
try:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="a") as writer:
        results_df[export_columns].to_excel(writer, sheet_name=sheet_name, index=False)
except FileNotFoundError:
    # If the file does not exist, create a new one
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="w") as writer:
        results_df[export_columns].to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Results successfully saved to {excel_filename} in sheet {sheet_name}.")

In [ ]:
RDII_combined_plot(all_params, delta_t, rainfall, area_acres, obs_rdii)

### Saving Metrices 

In [ ]:
criteria= calculate_criteria_multiple_runs(all_params, obs_rdii, delta_t, rainfall, area_acres)

In [ ]:
# Define the Excel file and sheet name
excel_filename = "Metrices_all_algorithms_Ro_E9_CCW.xlsx"
criteria_sheet_name = "CMA-ES"

# Select only the first 20 rows
criteria_df = criteria.iloc[:20].copy()

# Ensure "Run" column is present in criteria_df
if "Run" not in criteria_df.columns:
    criteria_df.insert(0, "Run", range(1, len(criteria_df) + 1))

# Add the "Fitness" column from results_df (first 20 rows)
criteria_df["Fitness"] = results_df.loc[:19, "Fitness"].values

# Select the required columns
export_columns_criteria = ["Run", "RMSE", "R2", "PBIAS", "NSE", "Fitness"]

# Check if the Excel file exists and append or create a new file
try:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="a") as writer:
        criteria_df[export_columns_criteria].to_excel(writer, sheet_name=criteria_sheet_name, index=False)
except FileNotFoundError:
    with pd.ExcelWriter(excel_filename, engine="openpyxl", mode="w") as writer:
        criteria_df[export_columns_criteria].to_excel(writer, sheet_name=criteria_sheet_name, index=False)

print(f"Criteria successfully saved to {excel_filename} in sheet {criteria_sheet_name}.")